## Visualize GLow results in 8+2 scenario

In [ ]:
from logging import WARNING, INFO
from typing import Callable, Dict, List, Optional, Tuple, Union
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from matplotlib import cm
import os

### Parse output data

In [ ]:
run_name = 'graph_8_32'
seed = '1999'
save_path = '../plots/'
#file_path = 'outputs/no_pretrain/' + 'seed_' + seed + '/' + run_name #NEED TO READ ALL FILES
root_path = '../outputs/GL/CIFAR10/seed1999_rounds1024/'
file_path = root_path + run_name
num_agents = 10

In [ ]:
tplgys = []
raw_tplgys = []
acc_distr = []
for i in range(5):
    # Read file
    file = open(file_path+'/'+str(i)+'_'+run_name+'_pool.out', mode = 'r', encoding = 'utf-8-sig')
    lines = file.readlines()
    tplgys.append(lines)
    file.close()

    # Read file
    file = open(file_path+'/'+str(i)+'_'+run_name+'_raw.out', mode = 'r', encoding = 'utf-8-sig')
    r_lines = file.readlines()
    raw_tplgys.append(r_lines)
    file.close()

    # Read file
    file = open(file_path+'/'+str(i)+'_'+run_name+'_acc_distr.out', mode = 'r', encoding = 'utf-8-sig')
    a_lines = file.readlines()
    acc_distr.append(a_lines)
    file.close()

In [ ]:
tplgy_ID = []
loss = []
acc = []

for tplgy in tplgys:
    tmp1 = []
    tmp2 = []
    tmp3 = []
    for line in tplgy:
        tmp1.append(int(line.split(' ')[1]))
        tmp2.append(float(line.split(' ')[-3]))
        tmp3.append(float(line.split(' ')[-1]))
    tplgy_ID.append(tmp1)
    loss.append(tmp2)
    acc.append(tmp3)
print(tplgy_ID)
print(loss)
print(acc)

In [ ]:
exec_times = []
for tplgy in raw_tplgys:
    exec_times.append(float(tplgy[-1].split(' ')[1])/60)
print(exec_times)

In [ ]:
# Mean accuracy of connected agents in topology 3
print('Mean: ' + str(np.mean(acc[3][:-2])))

In [ ]:
# Mean and var for accuracies in topoligies 2-4
tmp2 = []
for elems in acc[2:]:
    for e in elems[:-2]:
        tmp2.append(e)
print('Mean: ' + str(np.mean(tmp2)))
print('Var: ' + str(np.var(tmp2)))
#a = np.mean(acc[2:][:][:-2])
#print(a)

### Accuracies per node ID after *n* comm rounds

In [ ]:
font = {'size': 9}
plt.rc('font', **font)
viridis = cm.get_cmap('viridis', 5)
plt.plot(tplgy_ID[0], acc[0], '.', color=viridis(0))
plt.plot(tplgy_ID[1], acc[1], '.', color=viridis(0.2))
plt.plot(tplgy_ID[2], acc[2], '.', color=viridis(0.4))
plt.plot(tplgy_ID[3], acc[3], '.', color=viridis(0.6))
plt.plot(tplgy_ID[4], acc[4], '.', color=viridis(0.8))
plt.ylim(0, 1.)
x = np.arange(0, num_agents, 1)
plt.xticks(x)
#plt.title("Tplgy 0", fontsize=9)
plt.xlabel('AgentID')
plt.ylabel('Acc')
plt.grid()

ax = plt.gca()
box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])
lgd = ax.legend(['Tplgy 0', 'Tplgy 2', 'Tplgy 4', 'Tplgy 6', 'Tplgy 7'], loc='upper center',shadow=True, bbox_to_anchor=(0.5, -0.15), ncol = 5, fancybox= True)
plt.savefig(save_path+run_name+'_summarized_acc.pdf', bbox_extra_artists=(lgd,), bbox_inches='tight')

### Losses per node ID after *n* comm rounds

In [ ]:
font = {'size': 9}
plt.rc('font', **font)
viridis = cm.get_cmap('viridis', 5)
plt.plot(tplgy_ID[0], loss[0], '.', color=viridis(0))
plt.plot(tplgy_ID[1], loss[1], '.', color=viridis(0.2))
plt.plot(tplgy_ID[2], loss[2], '.', color=viridis(0.4))
plt.plot(tplgy_ID[3], loss[3], '.', color=viridis(0.6))
plt.plot(tplgy_ID[4], loss[4], '.', color=viridis(0.8))
x = np.arange(0, num_agents, 1)
plt.xticks(x)
plt.ylim(0, 250)
#plt.title("Tplgy 0", fontsize=9)
plt.xlabel('AgentID')
plt.ylabel('Loss')
plt.grid()

ax = plt.gca()
box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])

lgd = ax.legend(['Tplgy 0', 'Tplgy 2', 'Tplgy 4', 'Tplgy 6', 'Tplgy 7'], loc='upper center', shadow = True, bbox_to_anchor=(0.5, -0.15), ncol = 5, fancybox= True)

plt.savefig(save_path+run_name+'_summarized_loss.pdf', bbox_extra_artists=(lgd,), bbox_inches='tight')

### Loss evolution of each agent by topology

In [ ]:
#VISUALIZE LOSSES PER TOPOLOGY AND AGENT
loss_per_tplgy = []
num_rounds = 1024
for tplgy in raw_tplgys:
    tmp = tplgy[1].split()
    tmp_loss = []
    for i in range(1,num_rounds+1):
        tmp_loss.append((float(tmp[2*i][:-1])))
    loss_per_tplgy.append(tmp_loss)

In [ ]:
#VISUALIZE LOSSES PER TOPOLOGY AND AGENT
loss_per_tplgy_by_agent = []
num_rounds = 1024
for loss_tplgy in loss_per_tplgy:
    loss_per_agent = []
    for i in range(num_agents):
        tmp_loss = []
        for j in range(i+1, num_rounds, num_agents):
            tmp_loss.append((loss_tplgy[j]))
        loss_per_agent.append(tmp_loss)
    loss_per_tplgy_by_agent.append(loss_per_agent)

In [ ]:
font = {'size': 5}
plt.rc('font', **font)

#VISUALIZE LOSSES PER TOPOLOGY AND AGENT
viridis = cm.get_cmap('viridis', 10)
plasma = cm.get_cmap('plasma', 10)

fig, axs = plt.subplots(2, 2)
fig.tight_layout()

axs[0, 0].plot(loss_per_tplgy_by_agent[0][0][:101], color=plasma(0.8))
axs[0, 0].plot(loss_per_tplgy_by_agent[0][1][:101], color=viridis(0.))
axs[0, 0].plot(loss_per_tplgy_by_agent[0][2][:101], color=viridis(0.1))
axs[0, 0].plot(loss_per_tplgy_by_agent[0][3][:101], color=viridis(0.2))
axs[0, 0].plot(loss_per_tplgy_by_agent[0][4][:101], color=plasma(0.7))
axs[0, 0].plot(loss_per_tplgy_by_agent[0][5][:101], color=viridis(0.3))
axs[0, 0].plot(loss_per_tplgy_by_agent[0][6][:101], color=viridis(0.4))
axs[0, 0].plot(loss_per_tplgy_by_agent[0][7][:101], color=viridis(0.5))
axs[0, 0].plot(loss_per_tplgy_by_agent[0][8][:101], color=viridis(0.85))
axs[0, 0].plot(loss_per_tplgy_by_agent[0][9][:101], color=plasma(0.4))
axs[0, 0].grid()
axs[0, 0].set_ylim(0, 250)
#axs[0, 0].set_xlabel('Comm Round')
#axs[0, 0].set_ylabel('Loss')
axs[0, 0].set_title("Tplgy 0", fontsize=6)

axs[0, 1].plot(loss_per_tplgy_by_agent[1][0][:101], color=plasma(0.8))
axs[0, 1].plot(loss_per_tplgy_by_agent[1][1][:101], color=viridis(0.))
axs[0, 1].plot(loss_per_tplgy_by_agent[1][2][:101], color=viridis(0.1))
axs[0, 1].plot(loss_per_tplgy_by_agent[1][3][:101], color=viridis(0.2))
axs[0, 1].plot(loss_per_tplgy_by_agent[1][4][:101], color=plasma(0.7))
axs[0, 1].plot(loss_per_tplgy_by_agent[1][5][:101], color=viridis(0.3))
axs[0, 1].plot(loss_per_tplgy_by_agent[1][6][:101], color=viridis(0.4))
axs[0, 1].plot(loss_per_tplgy_by_agent[1][7][:101], color=viridis(0.5))
axs[0, 1].plot(loss_per_tplgy_by_agent[1][8][:101], color=viridis(0.85))
axs[0, 1].plot(loss_per_tplgy_by_agent[1][9][:101], color=plasma(0.4))
axs[0, 1].grid()
axs[0, 1].set_ylim(0, 250)
#axs[0, 1].set_xlabel('Comm Round')
#axs[0, 1].set_ylabel('Loss')
axs[0, 1].set_title("Tplgy 2", fontsize=6)

axs[1, 0].plot(loss_per_tplgy_by_agent[2][0][:101], color=plasma(0.8))
axs[1, 0].plot(loss_per_tplgy_by_agent[2][1][:101], color=viridis(0.))
axs[1, 0].plot(loss_per_tplgy_by_agent[2][2][:101], color=viridis(0.1))
axs[1, 0].plot(loss_per_tplgy_by_agent[2][3][:101], color=viridis(0.2))
axs[1, 0].plot(loss_per_tplgy_by_agent[2][4][:101], color=plasma(0.7))
axs[1, 0].plot(loss_per_tplgy_by_agent[2][5][:101], color=viridis(0.3))
axs[1, 0].plot(loss_per_tplgy_by_agent[2][6][:101], color=viridis(0.4))
axs[1, 0].plot(loss_per_tplgy_by_agent[2][7][:101], color=viridis(0.5))
axs[1, 0].plot(loss_per_tplgy_by_agent[2][8][:101], color=viridis(0.85))
axs[1, 0].plot(loss_per_tplgy_by_agent[2][9][:101], color=plasma(0.4))
axs[1, 0].grid()
axs[1, 0].set_ylim(0, 250)
#axs[1, 0].set_xlabel('Comm Round')
#axs[1, 0].set_ylabel('Loss')
axs[1, 0].set_title("Tplgy 4", fontsize=6)

axs[1, 1].plot(loss_per_tplgy_by_agent[4][0][:101], color=plasma(0.8))
axs[1, 1].plot(loss_per_tplgy_by_agent[4][1][:101], color=viridis(0.))
axs[1, 1].plot(loss_per_tplgy_by_agent[4][2][:101], color=viridis(0.1))
axs[1, 1].plot(loss_per_tplgy_by_agent[4][3][:101], color=viridis(0.2))
axs[1, 1].plot(loss_per_tplgy_by_agent[4][4][:101], color=plasma(0.7))
axs[1, 1].plot(loss_per_tplgy_by_agent[4][5][:101], color=viridis(0.3))
axs[1, 1].plot(loss_per_tplgy_by_agent[4][6][:101], color=viridis(0.4))
axs[1, 1].plot(loss_per_tplgy_by_agent[4][7][:101], color=viridis(0.5))
axs[1, 1].plot(loss_per_tplgy_by_agent[4][8][:101], color=viridis(0.85))
axs[1, 1].plot(loss_per_tplgy_by_agent[4][9][:101], color=plasma(0.4))
axs[1, 1].grid()
axs[1, 1].set_ylim(0, 250)
#axs[1, 1].set_xlabel('Comm Round')
#axs[1, 1].set_ylabel('Loss')
axs[1, 1].set_title("Tplgy 7", fontsize=6)

ax = plt.gca()
box = ax.get_position()
#ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])
lgd = ax.legend(['Agent 0 - ND', 'Agent 1 - R', 'Agent 2 - R', 'Agent 3 - R', 'Agent 4 - ND', 'Agent 5 - R', 'Agent 6 - R', 'Agent 7 - R', 'Agent 8 - D', 'Agent 9 - DND'], loc='upper center', shadow = True, bbox_to_anchor=(-0.15, -0.15), ncol = 5, fancybox= True)

plt.savefig(save_path+run_name+'_3_top_loss.pdf', bbox_extra_artists=(lgd,), bbox_inches='tight')
plt.show()

In [ ]:
#VISUALIZE LOSSES PER TOPOLOGY AND AGENT
viridis = cm.get_cmap('viridis', 10)
font = {'size': 9}
titles = ['Tplgy 0', 'Tplgy 2', 'Tplgy 4', 'Tplgy 6', 'Tplgy 7']


for i in range(5):
    plt.rc('font', **font)
    plt.plot(loss_per_tplgy_by_agent[i][0], color=plasma(0.8))
    plt.plot(loss_per_tplgy_by_agent[i][1], color=viridis(0.))
    plt.plot(loss_per_tplgy_by_agent[i][2], color=viridis(0.1))
    plt.plot(loss_per_tplgy_by_agent[i][3], color=viridis(0.2))
    plt.plot(loss_per_tplgy_by_agent[i][4], color=plasma(0.7))
    plt.plot(loss_per_tplgy_by_agent[i][5], color=viridis(0.3))
    plt.plot(loss_per_tplgy_by_agent[i][6], color=viridis(0.4))
    plt.plot(loss_per_tplgy_by_agent[i][7], color=viridis(0.5))
    plt.plot(loss_per_tplgy_by_agent[i][8], color=viridis(0.85))
    plt.plot(loss_per_tplgy_by_agent[i][9], color=plasma(0.6))
    plt.ylim(0, 250)
    plt.title(titles[i])
    plt.xlabel('Comm Round')
    plt.ylabel('Loss')
    ax = plt.gca()
    box = ax.get_position()
    ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])
    ax.legend(['Agent 0', 'Agent 1', 'Agent 2', 'Agent 3', 'Agent 4', 'Agent 5', 'Agent 6', 'Agent 7', 'Agent 8', 'Agent 9'], loc='center left', bbox_to_anchor=(1, 0.5))
    plt.grid()
    
    plt.savefig(save_path+str(i)+'_'+run_name+'_raw_summarized_loss.pdf')
    plt.show()


### Performance (Acc) evolution of each agent by topology

In [ ]:
'''Visualize acc average per pool and comm round'''
acc1 = []
for accuracies in acc_distr:
    acc2 = []
    for acc in accuracies:
        res = []
        for idx in acc.split(' '):
            f_idx = float(idx)
            if f_idx == 1.0:
                f_idx = 0.0
            res.append(f_idx)
        #res = [float(idx) for idx in acc.split(' ')]
        if res == 1.0:
            res = 0.0
        acc2.append(np.average(res))
    acc1.append(acc2)

In [ ]:
'''Create sets with exec times'''
acc_per_nodes = []
for i in range(num_agents):
    tmp_tplgy = []
    for j in range(5):
        tmp_tplgy.append(acc1[j][i::10])
    acc_per_nodes.append(tmp_tplgy)

In [ ]:
viridis = cm.get_cmap('viridis', 5)
font = {'size': 9}
for node in range(num_agents):
    plt.rc('font', **font)
    for i in range(5):
        plt.plot(acc_per_nodes[node][i], color=viridis(0.1 * 2.*i))
    #x = np.arange(0, num_rounds, 1)
    #plt.xticks(x)
    plt.ylim(0.0, 1.0)
    plt.title('Node ' + str(node))
    plt.xlabel('Comm Round')
    plt.ylabel('Acc')
    plt.grid()
    ax = plt.gca()
    box = ax.get_position()
    ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])
    ax.legend(['Tplgy 0', 'Tplgy 2', 'Tplgy 4', 'Tplgy 6', 'Tplgy 7'], loc='center left', bbox_to_anchor=(1, 0.5))
    plt.savefig(save_path+'node'+ str(node) +'_distr_val_acc.pdf')
    plt.show()

### Execution times

In [ ]:
'''Visualize Exec times per topology'''
font = {'size': 9}
plt.rc('font', **font)
x = [0, 2, 4, 6, 7]
plt.plot(x, exec_times, '.')
plt.xticks(x)
plt.title("Exec times " + run_name)
plt.xlabel("tplgy")
plt.ylabel("mins")
plt.grid()
plt.savefig(save_path+run_name+'_exec_times.pdf')
plt.show()